# Developing maximally-symmetric lattice population algorithm

In [ ]:
from polymerist.graphics.plotutils import presize_subplots

max_n_atoms : int = 10_000

records = {}
for mol_name, n_atoms in counts.items():
    n_oligomers = max_n_atoms // n_atoms
    cubic_lattice_dim = np.ceil(n_oligomers**(1/3))

    lattice_info = {
        'n_atoms'  : n_atoms,
        'n_oligomers' : n_oligomers,
        'occupancy' : n_oligomers / cubic_lattice_dim**3,
        'vacancy' : cubic_lattice_dim**3 - n_oligomers,
    }
    records[mol_name] = lattice_info

df = pd.DataFrame.from_dict(records, orient='index')

fig, ax = presize_subplots(nrows=2, ncols=2, scale=10, elongation=1.0)
for colname, axis in zip(df.columns, ax.flatten()):
    heights, bins, patches = axis.hist(df[colname])
    axis.set_xlabel(colname)
    axis.set_ylabel('Counts')

plt.show()

## Experimenting with RDKit conformers

In [ ]:
from rdkit import Chem
from rdkit.Chem.AllChem import EmbedMolecule
from rdkit.Chem import rdMolTransforms

In [ ]:
from polymerist.monomers import specification

smi = 'CC(=O)OC'
smi = specification.expanded_SMILES(smi, assign_map_nums=False)
rdmol = Chem.MolFromSmiles(smi, sanitize=False)
display(rdmol)

In [ ]:
err = EmbedMolecule(rdmol)
assert(err == 0)
display(rdmol)

In [ ]:
conf = rdmol.GetConformer(0)
pos  = conf.GetPositions() 
pos

# Playing with lattice generation (i.e. generating non-cubic lattices)

In [ ]:
from typing import Annotated, Any, Literal, Optional, TypeAlias, TypeVar
import numpy as np
import numpy.typing as npt
from polymerist.genutils.typetools.numpytypes import Shape, DType, N, M


def mean_coord(coords : np.ndarray[Shape[M, N], DType]) -> np.ndarray[Shape[N], DType]:
    '''Compute the average (center-of-mass) coordinate of a vector of coordinates'''
    return coords.mean(axis=0)
center_of_mass = COM = mean_coord

def dists_to_point(coords : np.ndarray[Shape[M, N], DType], point : np.ndarray[Shape[N], DType], norm_order : Optional[int]=None) -> np.ndarray[Shape[M], DType]:
    '''Compute the distance between each point in a coordinate array and a single arbitrary point'''
    return np.linalg.norm(coords - point, ord=norm_order, axis=1)

def dists_to_centroid(coords : np.ndarray[Shape[M, N], DType], norm_order : Optional[int]=None) -> np.ndarray[Shape[M], DType]:
    '''Compute the distance of each coordinate in an array of coordinates to the coordinates' centroid'''
    return dists_to_point(coords, point=mean_coord(coords), norm_order=norm_order)


def bounding_box(coords : np.ndarray[Shape[M, N], DType]) -> np.ndarray[Shape[2, N], DType]:
    '''Compute the tight bounding box around an array of coordinates'''
    return np.vstack([coords.min(axis=0), coords.max(axis=0)])

def bounding_box_dims(coords : np.ndarray[Shape[M, N], DType]) -> np.ndarray[Shape[N], DType]:
    '''Compute the tight bounding box dimensions around an array of coordinates'''
    return coords.ptp(axis=0)

In [ ]:
from dataclasses import dataclass, field
from polymerist.genutils.decorators.functional import flexible_listlike_input

import numpy as np
from polymerist.maths import lattices
from polymerist.maths.linearalg.decomposition import inv_left, inv_right
from polymerist.genutils.typetools.numpytypes import Shape, DType, N, M


class CubicIntegerLattice:
    '''For representing an n-dimensional integer lattice, consisting of all 
    n-tuples of integers with values constrained by side lengths in each dimension'''
    def __init__(self, sidelens : np.ndarray[Shape[N], int]) -> None: # TODO: implement more flexible input support (i.e. star-unpacking, listlikes, etc.)
        assert(sidelens.ndim == 1)
        self.sidelens = sidelens # ordered vector of the number of points along each dimension
        self.points : np.ndarray[Shape[M, N], int] = lattices.generate_int_lattice(*self.sidelens)

    def sidelens_as_str(self, multip_char : str='x') -> str:
        '''Stringify the lattice sidelengths'''
        return multip_char.join(str(i) for i in self.sidelens)

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}({self.n_dims}-dimensional, {self.sidelens_as_str()})'

    # LATTICE DIMENSIONS
    @property
    def n_dims(self) -> int: # referred to as "N" in typehints
        '''The number of dimensions of the lattice'''
        return self.sidelens.size
    
    @property
    def capacity(self) -> int: # referred to as "M" in typehints
        '''The maximum number of points that the lattice could contains'''
        return np.product(self.sidelens)

    @property
    def n_points(self) -> int: 
        '''The actual number of points currently contained in the lattice'''
        return np.product(self.sidelens)

    @property    
    def bounding_box_extrema(self) -> tuple[np.ndarray[Shape[N], int], np.ndarray[Shape[N], int]]:
        '''The minimal and maximal coordinates of the tight bounding box around the lattice'''
        return self.points.min(axis=0), self.points.max(axis=0)

    @property
    def bounding_box_sidelens(self) -> np.ndarray[Shape[N], int]:
        '''The side lengths of the tight bounding box around the points currently in the lattice'''
        return self.points.ptp(axis=0)
    
    # LEXICOGRAPHIC ORDERING
    @property
    def lex_ordered_weights(self) -> np.ndarray[Shape[N], int]:
        '''Vector of the number of points corresponding
        Can be viewed as a linear transformation between indices and point coordinates when coords are placed in lexicographic order'''
        return np.concatenate(([1], np.cumprod(self.sidelens)[:-1]))
    
    @property
    def lex_ordered_idxs(self) -> np.ndarray[Shape[M, N], int]:
        '''Returns a vector of the position that each point in self.points occupies when ordered lexicographically'''
        return np.lexsort(self.points.T)

    def lex_order_points(self) -> None:
        '''Sort points in the lattice in lexicographic order'''
        self.points = self.points[self.lex_order_idxs]

    # SUBLATTICE DECOMPOSITION
    @property
    def odd_even_idxs(self) -> tuple[np.ndarray[Shape[M], int], np.ndarray[Shape[M], int]]: # TOSELF: each subarray actually has length M/2 (+1 if capacity is odd), not sure how to typehint that though
        '''Return two vectors of indices, corresponding to points in the "odd" and "even" non-neighboring sublattices, respectively'''
        parity_vector = np.mod(self.points.sum(axis=1), 2) # remainder of sum of coordinates of each point; corresponds to the condition that a single step along any dimension should invert parity
        is_odd = parity_vector.astype(bool) # typecast as booleans to permit indexing (and make intent a bit clearer)

        return np.where(is_odd), np.where(~is_odd)

    @property
    def odd_idxs(self) -> np.ndarray[Shape[M], int]:
        '''Indices of the point in the in "odd" sublattice'''
        return self.odd_even_idxs[0]

    @property
    def even_idxs(self) -> np.ndarray[Shape[M], int]:
        '''Indices of the point in the in "even" sublattice'''
        return self.odd_even_idxs[1]

    @property
    def odd_sublattice(self) -> np.ndarray[Shape[M, N], int]:
        '''Returns points within the odd sublattice of the lattice points'''
        return self.points[self.odd_idxs]
    odd_points = odd_sublattice # alias for convenience

    @property
    def even_sublattice(self) -> np.ndarray[Shape[M, N], int]:
        '''Returns points within the even sublattice of the lattice points'''
        return self.points[self.even_idxs]
    even_points = even_sublattice # alias for convenience

    # LATTICE TRANSFORMATIONS
    def transformed_points(self, matrix : np.ndarray[Shape[N,N], float]) -> np.ndarray[Shape[M, N], float]:
        '''
        Accepts an NxN matrix (where N is the dimension of the lattice)
        Returns a linearly-transformed copy of the points currently in the lattice'''
        assert(matrix.shape == (self.n_dims, self.n_dims))
        return self.points @ matrix # NOTE: need to right-multiply multiply, since ROWS of self.points need to be tranformed 

In [69]:
%matplotlib inline
import matplotlib.pyplot as plt
# %matplotlib notebook

d = 3
dims = np.array([d, d, d])
latt = CubicIntegerLattice(dims)
latt


CubicIntegerLattice(3-dimensional, 3x3x3)

In [70]:
latt.even_points

AttributeError: 'CubicIntegerLattice' object has no attribute 'even_points'

In [ ]:

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter( *odd.T, color='b')
ax.scatter(*even.T, color='r')
plt.ion()
plt.show(block=True)

In [ ]:
plt.plot(range(10))
plt.show()

In [ ]:
plt.plot(range(10))
plt.show()

In [ ]:
odd = latt[odd_indices]
even = latt[even_indices]

### Dynamically registering common unit vector geenrators

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

A = np.array([
    [1, -1, 3],
    [1, 0, 5],
    [1, 2, 6],
])
R = np.array([
    [0, 1, -1],
    [1, 0, 0],
    [0, 1, 2],
])
sheared = latt @ A.T
sheared_redux = latt @ R.T

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(*sheared.T, color='b')
ax.scatter(*sheared_redux.T, color='r')

In [ ]:
sheared

In [ ]:
A_inv = np.linalg.inv(A)
inv_left(A) - A_inv, inv_right(A) - A_inv

# Playing with cubic/octahedral symmetry groups

In [ ]:
import numpy as np
from polymerist.graphics.plotutils import scatter_3D
from polymerist.maths.lattices import generate_int_lattice

d : int = 4

# produce integer lattice
dims = np.array([d, d, d])
num_points = dims.prod()
lattice = generate_int_lattice(*dims)

# sort lexicographically
order = np.lexsort(lattice.T)
lattice = lattice[order]

# determine even and odd positions after sorting
index_is_odd = lattice.sum(axis=1) % 2
index_is_odd = index_is_odd.astype(bool)

# center lattice at the origin
COM = lattice.mean(axis=0)
lattice = lattice - COM # sort lexicographically and translate center of grid to origin

# print(order, lattice[order])

In [ ]:
import matplotlib.pyplot as plt

alternate : bool = not False
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

if alternate:
    ax.scatter(*lattice[ index_is_odd].T, color='r')
    ax.scatter(*lattice[~index_is_odd].T, color='b')
else:
    ax.scatter(*lattice.T, color='r')

labels = np.lexsort(lattice.T)
for i, coords in zip(labels, lattice):
    ax.text(*coords, str(i))

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

plt.show()

In [ ]:
from typing import Generator, Iterable
from itertools import permutations, product as cartesian_product
from polymerist.maths.combinatorics.permutations import Permutation, Cycle
from scipy.spatial.transform import Rotation, Slerp


def sign_alternations(n : int) -> Generator[tuple[int, int, int], None, None]:
    '''Generate every possible n-tuple containing either 1 or -1'''
    for signs in cartesian_product(*([1, -1] for _ in range(n))):
        yield signs

def orthogonal_basis_transforms(n : int=3) -> Generator[np.ndarray, None, None]:
    '''Generate all matrices in n-dimensions which permute or invert the standard basis vectors'''
    for signs in sign_alternations(n):
        matr = np.diag(signs)
        for perm in Permutation.symmetric_group(n):
            yield matr @ perm.matrix

def interpolate_linear_transformation(trans : np.ndarray, n_frames : int=10) -> np.ndarray:
    '''Accepts a linear transformation matrix (of size MxN) and a number of frames F
    Returns an FxMxN array containing uniforms "steps" between the identity and the desired transformation'''
    I = Rotation.identity()
    rot = Rotation.from_matrix(trans)
    full_rot = Rotation.concatenate([I, rot])
    interpolator = Slerp(np.linspace(0, 1, num=len(full_rot), dtype=int), full_rot)

    return interpolator(np.linspace(0, 1, num=n_frames)).as_matrix()

In [ ]:
from collections import defaultdict

perms  = {}
all_cycles = {}

orbits = defaultdict(set)
stabilizers = defaultdict(list)

init_order = np.lexsort(lattice.T)
group_matrices = [el for el in orthogonal_basis_transforms(3)]
for i, trans in enumerate(group_matrices):
    new = lattice @ trans
    perms[i]      = perm   = Permutation(*np.lexsort(new.T))
    all_cycles[i] = cycles = perm.to_cycles(canonicalize=False)
    
    for cycle in cycles:
        if len(cycle) == 1:
            point_idx = cycle[0] # get the one element in the singleton cycle
            stabilizers[point_idx].append(trans)

        for point_idx in cycle:
            orbits[point_idx].update(cycle)

unique_orbits = set(tuple(sorted(orbit)) for orbit in orbits.values())
sum(len(i) for i in unique_orbits) == num_points

In [ ]:
stab_sizes = {k : len(v) for k, v in stabilizers.items()}
for i in unique_orbits:
    print([stab_sizes[j] for j in i])

## Visualize all symmetry actions as GIFs

In [ ]:
from pathlib import Path
from math import ceil
from matplotlib.animation import FuncAnimation, ArtistAnimation, writers
from rich.progress import track


n_frames = 20

dir = Path('gif_frames')
dir.mkdir(exist_ok=True)

ax_max = ceil(d / 2)
ax_min = -ax_max

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.autoscale(False)

def update(frame_no : int):
    partial_rot = keyframes[frame_no, :, :]
    latt_pos = lattice @ partial_rot

    ax.clear()
    ax.set(xlim3d=(ax_min, ax_max), xlabel='X')
    ax.set(ylim3d=(ax_min, ax_max), ylabel='Y')
    ax.set(zlim3d=(ax_min, ax_max), zlabel='Z')

    return ax.scatter(*latt_pos.T)

for i, trans in track(enumerate(group_matrices), total=len(group_matrices), description='Visualizing symmetry group actions...'):
    keyframes = interpolate_linear_transformation(trans, n_frames=n_frames)
    ani = FuncAnimation(fig=fig, func=update, frames=len(keyframes), interval=10)
    ani.save(dir/f'symmetry_{i+1}.gif', writer='pillow')
    plt.close(fig)

# Playing with rich progress

In [ ]:
from rich.progress import track, Progress
from time import sleep

with Progress() as progress:

    task1 = progress.add_task("[red]Downloading...", total=1000)
    task2 = progress.add_task("[green]Processing...", total=1000)
    task3 = progress.add_task("[cyan]Cooking...", total=1000)

    while not progress.finished:
        progress.update(task1, advance=0.5)
        progress.update(task2, advance=0.3)
        progress.update(task3, advance=0.9)
        sleep(0.02)

In [ ]:
import time
import random
from rich.progress import (
    BarColumn,
    Progress,
    SpinnerColumn,
    TaskProgressColumn,
    TimeElapsedColumn,
    TimeRemainingColumn,
)

def process(chunks):
    for chunk in chunks:
        time.sleep(0.1)
        yield chunk

chunks = [random.randint(1,20) for _ in range(100)]

progress_columns = (
    SpinnerColumn(),
    "[progress.description]{task.description}",
    BarColumn(),
    TaskProgressColumn(),
    "Elapsed:",
    TimeElapsedColumn(),
    "Remaining:",
    TimeRemainingColumn(),
)

with Progress(*progress_columns) as progress_bar:
    task = progress_bar.add_task("[blue]Downloading...", total=sum(chunks))
    for chunk in process(chunks):
        progress_bar.update(task, advance=chunk)

In [ ]:
import random
import time

from rich.live import Live
from rich.table import Table


def generate_table() -> Table:
    """Make a new table."""
    table = Table()
    table.add_column("ID")
    table.add_column("Value")
    table.add_column("Status")

    for row in range(random.randint(2, 6)):
        value = random.random() * 100
        table.add_row(
            f"{row}", f"{value:3.2f}", "[red]ERROR" if value < 50 else "[green]SUCCESS"
        )
    return table


with Live(generate_table(), refresh_per_second=4) as live:
    for _ in range(40):
        time.sleep(0.4)
        live.update(generate_table())

In [ ]:
from dataclasses import dataclass
from rich.console import Console, ConsoleOptions, RenderResult
from rich.table import Table

@dataclass
class Student:
    id: int
    name: str
    age: int
    def __rich_console__(self, console: Console, options: ConsoleOptions) -> RenderResult:
        yield f"[b]Student:[/b] #{self.id}"
        my_table = Table("Attribute", "Value")
        my_table.add_row("name", self.name)
        my_table.add_row("age", str(self.age))
        yield my_table